# ChromaDB Temel Test ve Prototip Raporu

**Oluşturan:** [Emre Yağız Koçak]
**Tarih:** 12.10.2025

### Amaç
Bu notebook, `chromadb` ve `langchain` kütüphanelerini kullanarak temel bir vektör veritabanı prototipi oluşturma adımlarını belgelemektedir. Amaç, veri ekleme ve anlamsal arama (semantic search) fonksiyonlarının çalıştığını doğrulamaktır. Bu script, daha sonra geliştirilecek olan `VectorStore` sınıfı için bir temel oluşturmuştur.

In [3]:
# Gerekli kütüphaneler
import chromadb
from typing import Optional
from langchain_core.documents import Document


# ChromaDB'ye bağlanmak için bir istemci (client)
client = chromadb.Client()

# 'test_collection' adında bir koleksiyon oluşturma
# Koleksiyonlar, verilerimizi gruplandırdığımız alanlardır.
collection = client.get_or_create_collection(name='test_collection')

print("ChromaDB istemcisi ve koleksiyonu başarıyla yüklendi.")

ChromaDB istemcisi ve koleksiyonu başarıyla yüklendi.


In [4]:
# LangChain'in Document nesnelerini kullanarak test verileri oluşturma
documents_to_add = [
    Document(
        page_content="Ben 21 yaşındayım.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Yapay zeka çok ilginç.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Ankara'da yaşıyorum.",
        metadata={"source": "mammal-pets-doc"},
    )
]

contents = [doc.page_content for doc in documents_to_add]
metadatas = [doc.metadata for doc in documents_to_add]
ids = [f"id{i+1}" for i in range(len(documents_to_add))]

# Sadece koleksiyon boşsa dokümanları ekle
# bu hücre tekrar çalıştırıldığında hatayı engeller.
if collection.count() == 0:
    print("Koleksiyon boş, dokümanlar ekleniyor...")
    collection.add(
        documents=contents,
        metadatas=metadatas,
        ids=ids
    )

# Ekleme işleminden sonra koleksiyondaki doküman sayısını kontrol etme kısmı.
print(f"Koleksiyonda toplam {collection.count()} adet doküman var.")

Koleksiyon boş, dokümanlar ekleniyor...
Koleksiyonda toplam 3 adet doküman var.


In [6]:
# Arama mantığı
def find_document(query: str) -> Optional[Document]:
    """
    Verilen bir sorguya göre koleksiyondaki en alakalı tek dokümanı
    içeriği ve metadatasıyla birlikte bir Document nesnesi olarak bulur.
    """
    results = collection.query(
        query_texts=[query],
        n_results=1,
        include=["metadatas", "documents"]
    )

    if results and results['documents'][0]:
        page_content = results['documents'][0][0]
        metadata = results['metadatas'][0][0]
        return Document(page_content=page_content, metadata=metadata)
    else:
        return None

print("`find_document` fonksiyonu tanımlandı.")

`find_document` fonksiyonu tanımlandı.


In [7]:
# örnek arama
search_query = "Yapay zeka uygulamaları"
found_doc = find_document(search_query)

print(f"'{search_query}' araması için bulunan sonuç:")

# Dönen sonucun bir Document nesnesi olup olmadığını kontrol edip detayları yazma kısmı.
if found_doc:
    print("Bulunan Döküman Tipi:", type(found_doc))
    print("Döküman İçeriği:", found_doc.page_content)
    print("Döküman Metadatası:", found_doc.metadata)
else:
    print("İlgili doküman bulunamadı.")

'Yapay zeka uygulamaları' araması için bulunan sonuç:
Bulunan Döküman Tipi: <class 'langchain_core.documents.base.Document'>
Döküman İçeriği: Yapay zeka çok ilginç.
Döküman Metadatası: {'source': 'mammal-pets-doc'}


Sonuc olarak test_RAG.py da kullanılacak fonksiyon eklendi.
